# Gather split questions into one `jsonl` file

structure:
```json
{
    "question": "",
    "idx": "",
    "paper": "",
    "answer: "",
}
```

In [1]:
# # generate blank files
# import os
# from datetime import datetime

# # set paper, 1 or 2
# paper = '2'

# cwd = os.getcwd()
# parent_dir = os.path.dirname(cwd)
# # set work dir
# current_year = str(datetime.now().year)
# work_dir = os.path.join(parent_dir, f'paper/{current_year}-{paper}')
# # find raw data, answer json
# src_dir = os.path.join(work_dir, "markdown")

# for lang_type in ["zh", "en"]:
#     for quiz_idx in range(1, 15):
#         fname = f"{quiz_idx:02}_{lang_type}.txt"
#         with open(os.path.join(src_dir, fname), 'a') as f:
#             pass

In [3]:
# write questions to json

import os
from datetime import datetime

import json

# set paper, 1 or 2
paper = '2'

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
# set work dir
current_year = str(datetime.now().year)
work_dir = os.path.join(parent_dir, f'paper/{current_year}-{paper}')
# find raw data, answer json
src_dir = os.path.join(work_dir, "markdown")

target_path = os.path.join(work_dir, f'question.jsonl')

questions = []

# read answer
answer_path = os.path.join(work_dir, 'answer.json')
with open(answer_path, 'r', encoding='utf-8') as f:
    answers = json.load(f)

# read value
value_path = os.path.join(work_dir, 'value.json')
with open(value_path, 'r', encoding='utf-8') as f:
    values = json.load(f)

# read question type
ques_type_path = os.path.join(work_dir, 'question_type.json')
with open(ques_type_path, 'r', encoding='utf-8') as f:
    ques_types = json.load(f)

for fname in os.listdir(src_dir):
    if not fname.endswith('txt'):
        continue

    ques_idx, lang_type = fname.replace('.txt', '').split('_')
    # remove the leading 0
    ques_idx = ques_idx.lstrip('0')
    ques_ans = answers[ques_idx]

    src_fpath = os.path.join(src_dir, fname)
    with open(src_fpath) as f:
        ques = f.readlines()
        ques = ''.join(ques)

    # add value
    value = values[ques_idx]

    # add question type
    # multip-choice: one correct
    if ques_idx in ['1', '2', '3', '4', '5', '6', '7', '8']:
        ques_type = '1'
    # multip-choice: multiple correct
    elif ques_idx in ['9', '10', '11']:
        ques_type = '2'
    # fill-in-the-blank
    elif ques_idx in ['12', '13', '14']:
        ques_type = '3'
    else:
        KeyError("only accept question index from 1 to 14.")


    tmp = {
            "question": ques,
            "idx": ques_idx,
            "paper": paper,
            "answer": ques_ans,
            "question_type": ques_type,
            "value": value,
            "lang": lang_type
        }

    questions.append(tmp)

with open(target_path, 'w', encoding='utf-8') as f:
    for entry in questions:
        # 将每个字典对象序列化为JSON字符串，并写入文件
        json_line = json.dumps(entry, ensure_ascii=False)
        tmp = f.write(json_line + '\n')
